# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 6 2023 8:30AM,256643,10,0086354865,ISDIN Corporation,Released
1,Feb 6 2023 8:30AM,256643,10,0086355122,ISDIN Corporation,Released
2,Feb 6 2023 8:30AM,256649,10,0086355123,ISDIN Corporation,Released
3,Feb 6 2023 8:30AM,256647,10,0086355126,ISDIN Corporation,Released
4,Feb 6 2023 8:30AM,256649,10,0086355124,ISDIN Corporation,Released
5,Feb 6 2023 8:30AM,256649,10,0086355125,ISDIN Corporation,Released
6,Feb 6 2023 8:30AM,256649,10,0086355293,ISDIN Corporation,Released
7,Feb 6 2023 8:30AM,256649,10,0086355294,ISDIN Corporation,Released
8,Feb 6 2023 8:30AM,256647,10,0086355302,ISDIN Corporation,Released
9,Feb 6 2023 8:30AM,256647,10,0086355301,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
7,256641,Released,1
8,256643,Released,2
9,256646,Released,4
10,256647,Released,16
11,256649,Released,17


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
256641,NaN,NaN,1.0
256643,NaN,NaN,2.0
256646,NaN,NaN,4.0
256647,NaN,NaN,16.0
256649,NaN,NaN,17.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256614,19.0,1.0,1.0
256618,0.0,0.0,5.0
256623,0.0,1.0,0.0
256630,0.0,0.0,12.0
256635,0.0,0.0,20.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
256614,19,1,1
256618,0,0,5
256623,0,1,0
256630,0,0,12
256635,0,0,20


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256614,19,1,1
1,256618,0,0,5
2,256623,0,1,0
3,256630,0,0,12
4,256635,0,0,20


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,256614,19,1,1
1,256618,,,5
2,256623,,1,
3,256630,,,12
4,256635,,,20


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation
2,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation
3,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation
25,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation
39,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC"
40,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation
60,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation
72,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc."
73,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation
78,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation,,,2
1,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation,,,17
2,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation,,,16
3,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation,,,4
4,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC",,,1
5,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,,,20
6,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation,,,12
7,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",,1,
8,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation,,,5
9,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",19,1,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation,2,,
1,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation,17,,
2,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation,16,,
3,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation,4,,
4,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC",1,,
5,Feb 3 2023 2:20PM,256635,10,ISDIN Corporation,20,,
6,Feb 3 2023 1:59PM,256630,10,ISDIN Corporation,12,,
7,Feb 3 2023 12:40PM,256623,12,"NBTY Global, Inc.",,1,
8,Feb 3 2023 12:25PM,256618,10,ISDIN Corporation,5,,
9,Feb 3 2023 11:19AM,256614,19,"AdvaGen Pharma, Ltd",1,1,19


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation,2,,
1,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation,17,,
2,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation,16,,
3,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation,4,,
4,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation,2.0,NaN,NaN
1,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation,17.0,NaN,NaN
2,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation,16.0,NaN,NaN
3,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation,4.0,NaN,NaN
4,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 6 2023 8:30AM,256643,10,ISDIN Corporation,2.0,0.0,0.0
1,Feb 6 2023 8:30AM,256649,10,ISDIN Corporation,17.0,0.0,0.0
2,Feb 6 2023 8:30AM,256647,10,ISDIN Corporation,16.0,0.0,0.0
3,Feb 6 2023 8:30AM,256646,10,ISDIN Corporation,4.0,0.0,0.0
4,Feb 3 2023 3:01PM,256641,19,"NAPP Technologies, LLC",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1796468,76.0,0.0,0.0
12,256623,0.0,1.0,0.0
19,513255,2.0,1.0,19.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1796468,76.0,0.0,0.0
1,12,256623,0.0,1.0,0.0
2,19,513255,2.0,1.0,19.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,76.0,0.0,0.0
1,12,0.0,1.0,0.0
2,19,2.0,1.0,19.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,12,Released,0.0
2,19,Released,2.0
3,10,Executing,0.0
4,12,Executing,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,19
Status,,,
Completed,0.0,0.0,19.0
Executing,0.0,1.0,1.0
Released,76.0,0.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,19
0,Completed,0.0,0.0,19.0
1,Executing,0.0,1.0,1.0
2,Released,76.0,0.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,19
0,Completed,0.0,0.0,19.0
1,Executing,0.0,1.0,1.0
2,Released,76.0,0.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()